# Web crawler - basic tools
<font size="1pt">This Jupyter notebook was created for the course Web Information Extraction and Retrieval at the University of Ljubljana. @szitnik</font>

## Fetching data from the Web

Python distribution already includes [*urllib* library](https://docs.python.org/3/library/urllib.html) which enables easy communication using HTTP requests. For those who would prefer a more feature-rich library we propose to take a look at the [*Requests* library](https://requests.readthedocs.io/en/master/). It is a high-level HTTP library and proposed to use by default for Python and supports multiple connections, sessions handling, proxies, etc. 

Let's retrieve [https://vreme.arso.gov.si](http://vreme.arso.gov.si) using a simple *urllib* library:

In [3]:
import urllib 

WEB_PAGE_ADDRESS = "https://vreme.arso.gov.si"

print(f"Retrieving web page URL '{WEB_PAGE_ADDRESS}'")

request = urllib.request.Request(
    WEB_PAGE_ADDRESS, 
    headers={'User-Agent': 'fri-ieps-TEST'}
)

with urllib.request.urlopen(request) as response: 
    html = response.read().decode("utf-8")
    print(f"Retrieved Web content: \n\n'\n{html}\n'")
    

Retrieving web page URL 'https://vreme.arso.gov.si'
Retrieved Web content: 

'
<!DOCTYPE html><html><head>
  <meta charset="utf-8">
  <title>ARSO vreme</title>
  <base href="/">

  <meta name="viewport" content="width=device-width, initial-scale=1">
  <link rel="icon" type="image/x-icon" href="favicon.ico">
  <style>html, 
body {
    height: 100%;
    margin:0;
}

.loading {
    background: #0082bc;
    color: #FFFFFF;
    width:100%;
    height:100%;
    margin:0;
}

.loading span {
    display:none;
    position:absolute;
    top:20px;
    left:20px;
    font-family: 'Open Sans', arial, helvetica, sans-serif;
}

.loading img {
    width:150px;
    height:22px;
    margin-left:100px;
    margin-top:50px;
}</style><link rel="stylesheet" type="text/css" href="default.css" media="print" onload="this.media='all'"><noscript><link rel="stylesheet" href="default.css"></noscript>
  <style>/* Slider */
</style><link rel="stylesheet" type="text/css" href="slick.css" media="print" onload="this.m

We can observe that the Web content we recieve is not expected. The HTML code includes a Javascript (JS) code that Web browser normally executes. The JS code above would *redirect* browser to the [*https://vreme.arso.gov.si/napoved*](https://vreme.arso.gov.si/napoved).

Libraries that enable us such functionality should automatically execute Javascript code. Generally, for this purpose they simulate browser such as Google Chrome or Firefox. An example of such library is [Selenium](https://www.selenium.dev/) ([Python API](https://selenium-python.readthedocs.io/)).

Selenium supports multiple browser drivers, so let's download and use a [GeckoDriver](https://github.com/mozilla/geckodriver/releases) (for Firefox). After that we can try to visit the ARSO Web page again but now using Selenium.

In [25]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.options import Options as FirefoxOptions

WEB_DRIVER_LOCATION = "../geckodriver-v0.34.0"
TIMEOUT = 5

firefox_options = FirefoxOptions()
# If you comment the following line, a browser will show ...
# firefox_options.add_argument("--headless")

#Adding a specific user agent
firefox_options.add_argument("user-agent=fri-ieps-TEST")

print(f"Retrieving web page URL '{WEB_PAGE_ADDRESS}'")
driver = webdriver.Firefox(executable_path=WEB_DRIVER_LOCATION, options=firefox_options)
driver.get(WEB_PAGE_ADDRESS)

# Timeout needed for Web page to render (read more about it)
time.sleep(TIMEOUT)

html = driver.page_source

print(f"Retrieved Web content (truncated to first 900 chars): \n\n'\n{html[:900]}\n'\n")

page_msg = driver.find_element(By.CSS_SELECTOR, ".panel-header")

print(f"Web page message: '{page_msg.text}'")

driver.close()

Retrieving web page URL 'https://vreme.arso.gov.si'
Retrieved Web content (truncated to first 900 chars): 

'
<html><head>
  <meta charset="utf-8">
  <title>ARSO vreme</title>
  <base href="/">

  <meta name="viewport" content="width=device-width, initial-scale=1">
  <link rel="icon" type="image/x-icon" href="favicon.ico">
  <style>html, 
body {
    height: 100%;
    margin:0;
}

.loading {
    background: #0082bc;
    color: #FFFFFF;
    width:100%;
    height:100%;
    margin:0;
}

.loading span {
    display:none;
    position:absolute;
    top:20px;
    left:20px;
    font-family: 'Open Sans', arial, helvetica, sans-serif;
}

.loading img {
    width:150px;
    height:22px;
    margin-left:100px;
    margin-top:50px;
}</style><link rel="stylesheet" type="text/css" href="default.css" media="all" onload="this.media='all'"><noscript><link rel="stylesheet" href="default.css"></noscript>
  <style>/* Slider */
</style><link rel="stylesheet" type="text/css" href="slick.css" media="all" on

Check also the *[WebDriverWait](https://en.wikipedia.org/wiki/Selenium_(software)#Selenium_WebDriver)* object to wait and check if the desired Web page loaded. Get familiar with different [options for locating elements](https://selenium-python.readthedocs.io/locating-elements.html).

The code above outputs a Deprecation Warning. You should udpate the code to use *[webdriver-manager](https://pypi.org/project/webdriver-manager/)* (Note that there is an issue for M1 processors architecture due to driver renaming). 

Note also that for MacOS you might allow execution of chromedriver/geckodriver in System Preferences.

## PostgreSQL database

### System initialization

In this tutorial we use PostgreSQL database Docker image (Windows users should use docker's IP instead of *localhost* below). You can also install the Database server locally to your machine.

First we prepare a file *database.sql*. The script will create a table with two rows:

```
CREATE SCHEMA IF NOT EXISTS showcase;

CREATE TABLE showcase.counters (
    counter_id integer  NOT NULL,
    value integer NOT NULL,
    CONSTRAINT pk_counters PRIMARY KEY ( counter_id )
 );

INSERT INTO showcase.counters VALUES (1,0), (2,0);
```

Go to an empty folder and save the script into a subfolder named *init_scripts*. Create another empty folder named *pgdata*.

We run docker container using the following command. The command will name the container *postgresql-wier*, set username and password, map database files to folder *./pgdata* and initialization scripts to *./init-scripts*, map port 5432 to host machine (i.e. localhost) and run image *postgres:9* in a detached mode. 

```
docker run --name postgresql-wier \
    -e POSTGRES_PASSWORD=SecretPassword \
    -e POSTGRES_USER=user \
    -v $PWD/pgdata:/var/lib/postgresql/data \
    -v $PWD/init-scripts:/docker-entrypoint-initdb.d \
    -p 5432:5432 \
    -d postgres:12.2
```

To check container's logs, run `docker logs -f postgresql-wier`.

To log into the database and execute SQL statements, run the following command: `docker exec -it postgresql-wier psql -U user`.

### Database example

Idea of the example below is to show how to implement concurrency and connect to a PostgreSQL database. 

We create three workers that run function *increase_db_values* and three workers that run function *increase_db_values_locking*. The first three workers will execute accesses to database interleavingly while the second three workers will enter into the database value update block in order.

In [24]:
import concurrent.futures
import threading
import psycopg2

lock = threading.Lock()
database_port = 5432

def reset_db_values():
    conn = psycopg2.connect(host="localhost", port = database_port, user="user", password="SecretPassword")
    conn.autocommit = True
    
    cur = conn.cursor()
    cur.execute("UPDATE showcase.counters SET value = 0")
    
    cur.close()
    conn.close()
    
def print_db_values():
    conn = psycopg2.connect(host="localhost", port = database_port, user="user", password="SecretPassword")
    conn.autocommit = True

    print("\nValues in the database:")
    cur = conn.cursor()
    cur.execute("SELECT counter_id, value FROM showcase.counters ORDER BY counter_id")
    for counter_id, value in cur.fetchall():
        print(f"\tCounter id: {counter_id}, value: {value}")
    cur.close()
    conn.close()

def increase_db_values(counter_id, increases):
    conn = psycopg2.connect(host="localhost", port = database_port, user="user", password="SecretPassword")
    conn.autocommit = True
    
    for i in range(increases):
        cur = conn.cursor()
        cur.execute("SELECT value FROM showcase.counters WHERE counter_id = %s", \
                    (counter_id,))
        value = cur.fetchone()[0]
        cur.execute("UPDATE showcase.counters SET value = %s WHERE counter_id = %s", \
                    (value+1, counter_id))
        cur.close()
    conn.close()
    
def increase_db_values_locking(counter_id, increases):
    conn = psycopg2.connect(host="localhost", port = database_port, user="user", password="SecretPassword")
    conn.autocommit = True
    
    for i in range(increases):
        with lock:
            cur = conn.cursor()
            cur.execute("SELECT value FROM showcase.counters WHERE counter_id = %s", \
                        (counter_id,))
            value = cur.fetchone()[0]
            cur.execute("UPDATE showcase.counters SET value = %s WHERE counter_id = %s", \
                        (value+1, counter_id))
            cur.close()
    conn.close()

reset_db_values()
print_db_values()

with concurrent.futures.ThreadPoolExecutor(max_workers=6) as executor:
    print(f"\n ... executing workers ...\n")
    for _ in range(3):
        executor.submit(increase_db_values, 1,1000)
    for _ in range(3):
        executor.submit(increase_db_values_locking, 2,1000)
    
print_db_values()


Values in the database:
	Counter id: 1, value: 0
	Counter id: 2, value: 0

 ... executing workers ...


Values in the database:
	Counter id: 1, value: 1015
	Counter id: 2, value: 3000
